Recommended to use https://tabula.technology/ instead of this script,
both work on same tabula, the above software provides more functionality and
GUI for better interface. If your PDF is based on image then recommended to use
Microsoft Office Mobile app functionality (Image to Table).

# Not Recommended

This notebook is more functional than the pdfTable2csv.py script, but is more Trial and Error based, so it is recommended to look into https://tabula.technology/

In [1]:
import tabula
import os, sys
import PyPDF2 as pdf2
import logging

# Info
page_no => go through each page of a file while testing

area => (top,left,bottom,right) # Trial And Error Method

colums => [end of 1st column, end of 2nd column] # divide right value (area) with no. of columns

lattice = True # True if table has border lines or else False

stream = False # True if table boder line are whitespace or else False

# Testing

In [2]:
file = r'C:\Users\atree\Desktop\SW_pdfs\Water_Quality_Drains_STPs__WTPs_2020.pdf'

In [7]:
page_no = 6
area = (92.9, 27.4, 828, 220)
col_div = [49,85,163.4,198.7,248.4,277.2,312.5,339.8,368.6,396.7,432.7,460.1,496.1,524.2,567.4,602.6,637.9,687.6,736.6,779.8]
lattice=True
stream=False

# ------------------------------------------------------
df= tabula.read_pdf(file, pages = page_no, area = area, columns = col_div, pandas_options={'header': None}, lattice=lattice, stream=stream)
df[0]

,0,1,2,3,4
0,24,20044,THANJAVUR,NaN,TAM
1,25,20045,TIRUNELVELI,NaN,TAM
2,26,20047,KUMBAKONAM,NaN,TAM
3,27,20048,SWARG ASHRAM - 1,NaN,UTT
4,29,20049,LAKKAR GHAT-\rOXIDATION PONDS,NaN,UTT
5,31,20050,JAGJITPUR,NaN,UTT


# Inputs

In [20]:
path = r'C:\Users\atree\Desktop\SW_pdfs\Water_Quality_data_of_Med_min_River_2020.pdf'
pages = [] # leave empty if all pages to be scanned
area_dict = {1:(147, 13, 774, 535), 'others':(147.6, 13, 774, 565)}
col_div_dict = {1:[48.2,207.4,283,312.5,342,368.6,396,421.2,448.6,481,510.5,537.1,565.2,594.7,624.2,655.2,694.8,730.1],
               'others': [49,205.9,283,312.5,342.7,368.6,396,421.2,449.3,479.5,510.5,536.4,565.9,595.4,625,655.2,694.8,730.1]}
lattice=True
stream=False

In [21]:
replaceDict = {}
splitDict = {}

In [22]:
outPath = os.path.join(os.path.expanduser("~"), 'Desktop', 'pdfTable2csv')
os.makedirs(outPath, exist_ok=True)
logging.basicConfig(filename=os.path.join(outPath,'pdfTable2csv.log'), format='%(asctime)s %(levelname)s: %(message)s', filemode='w', level=logging.DEBUG)

In [23]:
def extraction(file,page_no):
    if page_no in area_dict:
        area = area_dict[page_no]
    elif 'others' in area_dict:
        area = area_dict['others']
    elif 'all' in area_dict:
        area = area_dict['all']
    else:
        status = "area value cannot be fetched from area_dict"
        logging.error(status)
        raise ValueError(status)
    logging.info('area value fetched from area_dict')
    if page_no in col_div_dict:
        col_div = col_div_dict[page_no]
    elif 'others' in col_div_dict:
        col_div = col_div_dict['others']
    elif 'all' in col_div_dict:
        col_div = col_div_dict['all']
    else:
        status = "column division value cannot be fetched from col_div_dict"
        logging.error(status)
        raise ValueError(status)
    try:
        table = tabula.read_pdf(file, pages = page_no, area = area, columns = col_div, pandas_options={'header': None}, lattice=lattice, stream=stream)[0]
    except Exception as e:
        status = "failed reading pdf using tabula-py"
        logging.exception(f'{e}: {status}')
        raise Exception(status)
    filename = os.path.split(file)[1].split('.')[0]
    logging.info(f"filename acquired: {filename}")
    if replaceDict:
        for present_ele, new_ele in replaceDict.items():
            table = table.apply(lambda col: col.astype(str).str.replace(present_ele, new_ele))
            logging.info("Values manipulated by replace method")
    else:
        logging.info('No manipulation by replace method')
    if splitDict:
        for column_num, condition in splitDict.items():
            split_ele, position = condition
            table.iloc[:, column_num] = table.iloc[:, column_num].astype(str).str.split(split_ele).apply(lambda x: x[position])
            logging.info(f"Column {column_num} values got truncated.")
    else:
        logging.info('no spliting of values operation made.')
    outFile = os.path.join(outPath, f'{filename}_{page_no}.csv')
    table.to_csv(outFile)
    logging.info(f'table saved to file {outFile}')

In [24]:
def loop_pages(file, pages, count=1, totalFiles=1):
    if pages:
        totalpages = len(pages)
    else:
        try:
            with open (file, "rb") as f:
                pdf = pdf2.PdfFileReader(f)
                totalpages = pdf.numPages
            pages = range(1,totalpages+1)
        except Exception as e:
            status = "failed reading total number of pages using PyPDF2."
            logging.exception(f'{e}: {status}')
            raise Exception(status)
    pg_count = 1
    for page_no in pages:
        logging.info(f'Started Page: {page_no}')
        extraction(file,page_no)
        progress = pg_count/totalpages
        block = int(round(10*progress))
        progress_text = "\r{0}: [{1}] {2} {3:06.2f}%".format("Pages", "#"*block + "-"*(10-block), 'Files:', (count/totalFiles)*100)
        sys.stdout.write(progress_text)
        sys.stdout.flush()
        pg_count += 1

In [25]:
print(f'Output files saved at {outPath}')
if os.path.splitext(path)[1]:
    logging.info(f'Single PDF file: {path}')
    loop_pages(path, pages)
else:
    logging.info(f'Directory Provided: {path}')
    for _, _, files in os.walk(path):
        totalFiles = len(files)
        count=1
        for file in files:
            if os.path.splitext(file)[1] == '.pdf':
                logging.info(f'Started: {file}')
                loop_pages(os.path.join(path, file), pages, count, totalFiles)
            count+=1

Output files saved at C:\Users\atree\Desktop\pdfTable2csv
Pages: [##########] Files: 100.00%